# **Tarea Nº 4**

- Fecha y hora de entrega: Domingo 12 de enero, 11:59pm
- Agregue los nombres de las personas con las que discutió esta tarea: __Rodrigo Farfan__
- Envíe su tarea haciendo el `push` de su código a su repo en GitHub Classroom: 

### Evaluación

- Nota: **17/20**

¡Buen trabajo en general!

## **Mercado de autos usados**

Importa el conjunto de datos `neoauto_20240924.csv`
    

1. ¿Cuál es el precio promedio de los autos de cada marca (`item_brand`) publicados en cada subcategoría (`item_category_2`)?

In [7]:
# Solución

import pandas as pd

dfautos = pd.read_csv("neoauto_20240924.csv")

promedio_precio = dfautos.groupby(['item_brand', 'item_category_2'])['item_price'].mean().reset_index()

# Mostrar la información obtenida en un dataframe nuevo
print(promedio_precio)

       item_brand item_category_2     item_price
0      ALFA-ROMEO       Hatchback    8500.000000
1      ALFA-ROMEO           Sedan    2500.000000
2    ASTON MARTIN  Camionetas Suv  229000.000000
3    ASTON MARTIN       Deportivo  169990.000000
4    ASTON MARTIN           Sedan  149900.000000
..            ...             ...            ...
197         VOLVO  Camionetas Suv   41801.707317
198         VOLVO       Deportivo    8800.000000
199         VOLVO       Hatchback   12772.500000
200         VOLVO           Sedan   11458.333333
201         VOLVO   Station Wagon    6100.000000

[202 rows x 3 columns]


2. ¿Cuántos autos tienen más de 100,000 kilómetros (`item_km`) pero un precio (`item_price`) menor al promedio general?

In [9]:
# Solución

precio_promedio = dfautos['item_price'].mean() # Promedio general

autos_filtrados = dfautos[(dfautos['item_km'] > 100000) & (dfautos['item_price'] < precio_promedio)] # Haciendo el filtro

numero_autos = autos_filtrados.shape[0] # Hallando el número

print(numero_autos)

497


3. Encuentra el modelo (`item_name`) más caro y el más barato de cada marca (`item_brand`).

In [11]:
# Solución

# Hallando los máximos y mínimos de cada marca y anexándolos a nuevos dataframe
max_precio = dfautos.loc[dfautos.groupby('item_brand')['item_price'].idxmax()][['item_brand', 'item_name', 'item_price']]
min_precio = dfautos.loc[dfautos.groupby('item_brand')['item_price'].idxmin()][['item_brand', 'item_name', 'item_price']]

# Unir los dataframes para obtener el resultado final
resultados = pd.merge(max_precio, min_precio, on='item_brand', suffixes=('_max', '_min'))

print(resultados)


      item_brand             item_name_max  item_price_max  \
0     ALFA-ROMEO       alfa-romeo 147 2006          8500.0   
1   ASTON MARTIN     aston martin dbx 2021        229000.0   
2           AUDI            audi rs 6 2024        220000.0   
3         AUSTIN   austin mini cooper 1970             1.0   
4           BAIC             baic x55 2021         12800.0   
..           ...                       ...             ...   
68           SWM          swm g05 pro 2022         14000.0   
69        TOYOTA         toyota hilux 2019        100000.0   
70       TRIUMPH         triumph tr-7 1971         18900.0   
71    VOLKSWAGEN  volkswagen teramont 2023         48900.0   
72         VOLVO          volvo xc-90 2023         84990.0   

                     item_name_min  item_price_min  
0          alfa-romeo alfasud 1982          2500.0  
1           aston martin db11 2017        149900.0  
2               audi a3 sedan 2015             0.0  
3          austin mini cooper 1970         

4. Encuentra las tres marcas con mayor número de autos financiados por Santander (`item_financed_by`). 

In [12]:
# Solución

autos_financiados_santander = dfautos[dfautos['item_financed_by'] == 'Santander']

conteo_marcas = autos_financiados_santander['item_brand'].value_counts()

print(conteo_marcas.head(3))

item_brand
BMW       296
TOYOTA    261
NISSAN    228
Name: count, dtype: int64


5. Agrupa las publicaciones por tipo de transmisión (`item_transmission`) y calcula la desviación estándar del precio (`item_price`) para cada tipo.  

In [15]:
# Solución

dfautos.groupby('item_transmission')['item_price'].std().reset_index().rename(columns={'item_price': 'sd_itemprice'})

,item_transmission,sd_itemprice
0,Automática,15332.729048
1,Automática - Secuencial,22678.641589
2,Mecánica,8137.122127


6. ¿Cuál es el kilometraje promedio de las publicaciones con el tag "Premium" y de las publicaciones con el tag "Como nuevo"?

In [17]:
# Solución
dfautos[dfautos['item_tag'].isin(['Premium', 'Como nuevo'])].groupby('item_tag')['item_km'].mean().reset_index()

,item_tag,item_km
0,Como nuevo,35963.476190
1,Premium,48400.235897


7. ¿Cuáles son las marcas de vehículas para los que todos sus modelos fabricados en los últimos 5 años tienen al menos 5 publicaciones cada una?

In [20]:
# Solución

from datetime import datetime

anio_actual = datetime.now().year

dfautos_recientes = dfautos[dfautos['item_year'] >= anio_actual - 5]

conteo_publicaciones = dfautos_recientes.groupby(['item_brand', 'item_name']).size()

modelos_filtrados = conteo_publicaciones[conteo_publicaciones >= 5]

marcas_validas = []
for marca, grupo in dfautos_recientes.groupby('item_brand'):
    
    modelos_marca = grupo['item_name'].unique()
    publicaciones_modelos = conteo_publicaciones[conteo_publicaciones.index.get_level_values('item_brand') == marca]
    
    if all(publicaciones_modelos >= 5):
        marcas_validas.append(marca)

print(marcas_validas)

[]


## **Evolución económica internacional**

Importa el conjunto de datos de la hoja "Data" del archivo `pwt1001.xlsx`. El diccionario de variables está disponible en la hoja "Legend" del archivo Excel.
    

1. Calcule el PBI per cápita real ajustado por poder de paridad de compra (PPP). Use la variable de PBI que se calcula por gasto y la población. ¿Cuáles son los 10 países top?

In [1]:
# Solución

# Cargamos los datos
import pandas as pd
df = pd.read_excel("pwt1001.xlsx", sheet_name="Data")

# Calcular el PPP
df['ppp'] = df['rgdpe'] / (df['pop'])

#tomar el ultimo año
df = df.sort_values('year', ascending=False).drop_duplicates('countrycode')

# Ordenar los 10 primeros
df_top10 = df.sort_values('ppp', ascending=False).head(10)     
print(df_top10[['country', 'ppp']])

                    country            ppp
7209             Luxembourg  112941.453342
7349       China, Macao SAR  105337.890061
9799                  Qatar  103445.127269
5599                Ireland  102353.631443
10289             Singapore   88619.305271
1609                Bermuda   79967.588736
2239            Switzerland   71831.637057
3079         Cayman Islands   70208.345162
349    United Arab Emirates   69753.215256
1889      Brunei Darussalam   67555.392597


2. Genera el ránking de los países con el mayor incremento del PBI per cápita real PPP entre los años 1990 y 2019. ¿Dónde se ubica Perú? Para los países que no tengan información para dichos años, puede utilizar datos provenientes de hasta dos años antes o después

In [2]:
# Solución
df_2 = pd.read_excel("pwt1001.xlsx", sheet_name="Data")
df_2 = df_2.fillna(method="bfill")
df_2['pbi_per_capita_1'] = df_2['rgdpe'] / df_2['pop']

# Filtrar datos
df_1990 = df_2[df_2['year'] == 1990][['country', 'pbi_per_capita_1']].rename(columns={"pbi_per_capita_1": "pbi_per_capita_1990"})
df_2019 = df_2[df_2['year'] == 2019][['country', 'pbi_per_capita_1']].rename(columns={"pbi_per_capita_1": "pbi_per_capita_2019"})

# Combinar datos
df_2 = pd.merge(df_1990, df_2019, on="country", how="inner")

# Incremento
df_2['incremento_pbi_per_capita'] = df_2['pbi_per_capita_2019'] - df_2['pbi_per_capita_1990']

# Ordenar por el mayor incremento
df_2 = df_2.sort_values(by="incremento_pbi_per_capita", ascending=False)
df_2.head(10)


C:\Users\Jose Antonio\AppData\Local\Temp\ipykernel_13508\2262902863.py:3: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_2 = df_2.fillna(method="bfill")


,country,pbi_per_capita_1990,pbi_per_capita_2019,incremento_pbi_per_capita
79,Ireland,19626.917875,102353.631443,82726.713568
104,"China, Macao SAR",26549.065903,105337.890061,78788.824158
139,Qatar,28000.258728,103445.127269,75444.868541
146,Singapore,21527.149763,88619.305271,67092.155508
102,Luxembourg,45944.414778,112941.453342,66997.038564
22,Bermuda,31735.779793,79967.588736,48231.808944
112,Malta,14790.949022,50759.083017,35968.133995
127,Norway,28445.726947,62544.032357,34098.305410
94,Kuwait,21887.202650,55942.795608,34055.592958
143,Saudi Arabia,18205.485504,51199.350465,32993.864961


3. Para el periodo 2010-2019, calcule el promedio del índice del capital humano. ¿Qué relación existe entre esta variable y el PBI per cápita real PPP calculado previamente? ¿y el número de horas laboradas al año por los trabajadores?

In [3]:
# Solución
df_2 = pd.read_excel("pwt1001.xlsx", sheet_name="Data")

# Filtrar los datos
df_2_filt = df_2[(df_2['year'] >= 2010) & (df_2['year'] <= 2019)].copy()

# Manejar missings
df_2_filt['hc'] = df_2_filt['hc'].fillna(df_2_filt['hc'].mean())
df_2_filt['avh'] = df_2_filt['avh'].fillna(df_2_filt['avh'].mean())

# Calcular PPP
df_2_filt.loc[:, 'pib_per_capita_ppp'] = df_2_filt['rgdpe'] / df_2_filt['pop']

# Calcular promedios
promedio_hc = df_2_filt.groupby('country')['hc'].mean().reset_index()
promedio_hc.rename(columns={'hc': 'promedio_hc_2010_2019'}, inplace=True)
promedio_pib = df_2_filt.groupby('country')['pib_per_capita_ppp'].mean().reset_index()
promedio_horas = df_2_filt.groupby('country')['avh'].mean().reset_index()

# Combinar todo
relacion = promedio_hc.merge(promedio_pib, on='country').merge(promedio_horas, on='country')

# Renombrar columnas
relacion.rename(columns={'pib_per_capita_ppp': 'promedio_pib_per_capita_ppp',
                         'avh': 'promedio_horas_laboradas'}, inplace=True)

#Primeros resultados
print(relacion.head(10))

# Explorar relación
correlaciones = relacion[['promedio_hc_2010_2019', 'promedio_pib_per_capita_ppp', 'promedio_horas_laboradas']].corr()
print(correlaciones)

               country  promedio_hc_2010_2019  promedio_pib_per_capita_ppp  \
0              Albania               2.934304                 11029.751620   
1              Algeria               2.207947                 12797.237930   
2               Angola               1.449344                  7627.644937   
3             Anguilla               2.608130                 27167.015652   
4  Antigua and Barbuda               2.608130                 20046.143125   
5            Argentina               2.961662                 21492.060853   
6              Armenia               3.123577                 10879.322226   
7                Aruba               2.608130                 38145.486904   
8            Australia               3.489004                 49392.158049   
9              Austria               3.331753                 51477.323055   

   promedio_horas_laboradas  
0               1873.344553  
1               1873.344553  
2               1873.344553  
3               1873.

4. A partir de la variable del precio de los hogares, calcula los países y los años en los cuales se registró la mayor inflación anual para todo el periodo de tiempo disponible en la base de datos

In [18]:
# Solución

# Calcular la inflación anual
df_2['inflacion_anual'] = df_2.groupby('country')['pl_con'].pct_change(fill_method=None) * 100

# Filtrar los países y años con la mayor inflación
mayor_inf = df_2.loc[df_2.groupby('country')['inflacion_anual'].idxmax(), ['country', 'year', 'inflacion_anual']]

# Ordenar
mayor_inf = mayor_inf.sort_values(by='inflacion_anual', ascending=False)
print(mayor_inf)


                                  country  year  inflacion_anual
12387  Venezuela (Bolivarian Republic of)  2017       360.890723
4742                    Equatorial Guinea  2002       174.837577
12795                            Zimbabwe  2005       166.320309
304                  United Arab Emirates  1974       148.351099
10034                        Saudi Arabia  1974       146.868541
...                                   ...   ...              ...
12134                       United States  1974        10.113069
11181            Turks and Caicos Islands  2001         8.606866
58                                  Aruba  2008         7.424463
2998                              Curaçao  2008         5.958649
10989           Sint Maarten (Dutch part)  2019         5.164641

[183 rows x 3 columns]


5. ¿Cuáles son los países con la mayor cantidad de missing values en la variable de productividad total de factores a precios corrientes?

In [17]:
# Solución
missing = df_2.groupby('country')['ctfp'].apply(lambda x: x.isna().sum()).reset_index()

# Renombrar columna
missing.rename(columns={'ctfp': 'missing_count'}, inplace=True)

# Ordenar los países
missing = missing.sort_values(by='missing_count', ascending=False)

# Mostrar los países
print(missing.head(10))

                   country  missing_count
0                  Albania             70
1                  Algeria             70
3                 Anguilla             70
4      Antigua and Barbuda             70
13              Bangladesh             70
10              Azerbaijan             70
7                    Aruba             70
22  Bosnia and Herzegovina             70
20                  Bhutan             70
17                  Belize             70


6.  Agrupa los datos por país y calcula el promedio de las horas trabajadas anuales (avh) y de la educación promedio (hc) para el periodo 2000-2020.

In [9]:
# Solución

df_periodo_2000_2020 = df_2[(df_2['year'] >= 2000) & (df_2['year'] <= 2020)]

average_data = df_periodo_2000_2020.groupby('country', as_index=False).agg(
    avg_hours_worked=('avh', 'mean'),
    avg_education=('hc', 'mean')
)

df_periodo_2000_2020.head()

,countrycode,country,currency_unit,year,rgdpe,rgdpo,pop,emp,avh,hc,...,csh_m,csh_r,pl_c,pl_i,pl_g,pl_x,pl_m,pl_n,pl_k,inflacion_anual
50,ABW,Aruba,Aruban Guilder,2000,4031.134277,4129.512695,0.090853,0.041900,NaN,NaN,...,-0.675893,-0.022756,0.544148,0.364808,0.569621,0.421769,0.479504,0.339565,0.830465,4.621669
51,ABW,Aruba,Aruban Guilder,2001,4122.958496,4728.453125,0.092898,0.042808,NaN,NaN,...,-0.626023,-0.010782,0.548881,0.336240,0.608278,0.377714,0.466912,0.315414,0.821823,2.322797
52,ABW,Aruba,Aruban Guilder,2002,4101.703125,3555.261230,0.094992,0.043736,NaN,NaN,...,-0.819539,-0.003255,0.568648,0.336211,0.615146,0.503704,0.463264,0.318944,0.734359,2.990286
53,ABW,Aruba,Aruban Guilder,2003,4209.140137,3701.983643,0.097017,0.044632,NaN,NaN,...,-0.795562,-0.002240,0.591519,0.346326,0.584422,0.532451,0.491250,0.315550,0.672940,1.833795
54,ABW,Aruba,Aruban Guilder,2004,4548.991211,4227.812012,0.098737,0.045385,NaN,NaN,...,-0.743200,-0.008779,0.608234,0.355328,0.591757,0.517250,0.501850,0.318468,0.648683,2.462345


7.  Genere una variable que ordene a los países según el porcetaje de formación brutal de capital del 2019. Luego, estable diez grupos a manera de deciles. ¿En qué decil se ubica Péru? Vuelva a calcular usando la información de 1990. ¿El Perú se mantiene en el mismo décil o cambia?

In [16]:
# Solución

# Filtrar datos para 2019
df_2019 = df_2[df_2['year'] == 2019].copy()
df_2019['deciles_2019'] = pd.qcut(df_2019['csh_i'], 10, labels=False) + 1
peru_decil_2019 = df_2019[df_2019['country'] == 'Peru']['deciles_2019'].values[0] if 'Peru' in df_2019['country'].values else None

# Filtrar datos para 1990 
df_1990 = df_2[df_2['year'] == 1990].copy()
df_1990['deciles_1990'] = pd.qcut(df_1990['csh_i'], 10, labels=False) + 1
peru_decil_1990 = df_1990[df_1990['country'] == 'Peru']['deciles_1990'].values[0] if 'Peru' in df_1990['country'].values else None

# Resultados
print(f"Decil 1990: {peru_decil_1990}, Decil 2019: {peru_decil_2019}")


Decil 1990: 4.0, Decil 2019: 5


## **Transacciones Financieras**

Importa el conjunto de datos `base_financiera.csv`
    

8. Filtra todas las transacciones de tipo "Debit" en las que el monto sea mayor a 200, y calcula el monto total de esas transacciones. Además, determina cuántos clientes únicos realizaron esas transacciones.

In [10]:
# Solución

import pandas as pd

dffinance = pd.read_csv("base_financiera.csv")

transacciones_debit = dffinance[(dffinance['Transaction_Type'] == 'Debit') & (dffinance['Amount'] > 200)]

# Calculamos el monto total de esas transacciones
monto_total = transacciones_debit['Amount'].sum()

# Determinamos cuántos clientes únicos realizaron esas transacciones
clientes_unicos = transacciones_debit['Customer_ID'].nunique()

# Mostrar los resultados
print(f'Monto total de las transacciones: {monto_total}')
print(f'Número de clientes únicos: {clientes_unicos}')

Monto total de las transacciones: 2847871.75
Número de clientes únicos: 200


9. Identifica las transacciones con valores nulos en la columna (`Amount`) . Rellena esos valores con la mediana de los montos por tipo de transacción (`Transaction_Type`) y luego calcula el monto total de transacciones por región (`region`) .

In [13]:
# Solución

import pandas as pd

transacciones_nulas = dffinance[dffinance['Amount'].isnull()]

medianas_por_tipo = dffinance.groupby('Transaction_Type')['Amount'].median()

# Función para rellenar los valores nulos con la mediana correspondiente
def rellenar_nulos(row):
    if pd.isnull(row['Amount']):
        return medianas_por_tipo[row['Transaction_Type']]
    else:
        return row['Amount']

dffinance['Amount'] = dffinance.apply(rellenar_nulos, axis=1)

monto_total_por_region = dffinance.groupby('Region')['Amount'].sum().reset_index()

print("\nMonto total de transacciones por región:")
print(monto_total_por_region)


Monto total de transacciones por región:
  Region      Amount
0   East  1269409.53
1  North  1321224.96
2  South  1208300.23
3   West  1233552.50


10. Busca transacciones duplicadas considerando las columnas (`Customer_ID`) y (`Date`). Elimina los duplicados manteniendo solo la primera ocurrencia. Luego, agrupa las transacciones restantes por (`Customer_ID`) y calcula el monto promedio y la cantidad de transacciones por cliente.

In [20]:
# Solución

dffinance_sin_duplicados = dffinance.drop_duplicates(subset=['Customer_ID', 'Date'], keep='first')

resultados = dffinance_sin_duplicados.groupby('Customer_ID').agg(
    monto_promedio=('Amount', 'mean'),
    cantidad_transacciones=('Amount', 'size')
).reset_index()

print(resultados)

    Customer_ID  monto_promedio  cantidad_transacciones
0          C001     3283.319412                      17
1          C002     1881.827778                       9
2          C003     3102.125000                      14
3          C004     2879.267143                       7
4          C005     3364.053333                       6
..          ...             ...                     ...
195        C196     2566.555000                      10
196        C197     2530.674286                      14
197        C198     2484.925000                      12
198        C199     2942.038000                       5
199        C200     2196.990000                       5

[200 rows x 3 columns]
